<a href="https://colab.research.google.com/github/ddopazo92/Introduccion_datascience_diego_dopazo/blob/main/Proyecto_final_dopazo_diego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Diagrama de dispersion entre ventas de juegos y puntuacion promedio  donde el resultado me va a decir que consola es mas probable que saque un juego con una punutuacion mas alta y que sea mas popular

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [15]:
url = ("https://raw.githubusercontent.com/ddopazo92/Introduccion_datascience_diego_dopazo/refs/heads/main/Video_Games.csv")
df = pd.read_csv(url, sep=';')

In [19]:
##REEMPLAZO DE PALABRAS EN INGLES A ESPAÑOL
df = df.set_axis(['Juego', 'Consola', 'Año de lanzamiento','Genero', 'Publicador', 'Ventas USA','Ventas Europa', 'Ventas Japon','Ventas resto del Mundo',
                  'Ventas Totales', 'Puntuacion de criticos','Cantidad de criticos', 'Puntaje de Usuario','Cantidad de usuarios','Desarrollador','Clasificacion_por_Edad'], axis=1)

df['Juego'] = df['Juego'].str.upper()
df['Genero'] = df['Genero'].str.replace('Action','Accion')
df['Genero'] = df['Genero'].str.replace('Adventure','Aventura')
df['Genero'] = df['Genero'].str.replace('Fighting','Lucha')
df['Genero'] = df['Genero'].str.replace('Misc','Miscelania')
df['Genero'] = df['Genero'].str.replace('Platform','Plataforma')
df['Genero'] = df['Genero'].str.replace('Racing','Carreras')
df['Genero'] = df['Genero'].str.replace('Role-Playing','RPG')
df['Genero'] = df['Genero'].str.replace('Shooter','Disparos')
df['Genero'] = df['Genero'].str.replace('Simulation','Simulacion')
df['Genero'] = df['Genero'].str.replace('Sports','Deportes')
df['Genero'] = df['Genero'].str.replace('Strategy','Estrategia')

In [20]:
df.head()

,Juego,Consola,Año de lanzamiento,Genero,Publicador,Ventas USA,Ventas Europa,Ventas Japon,Ventas resto del Mundo,Ventas Totales,Puntuacion de criticos,Cantidad de criticos,Puntaje de Usuario,Cantidad de usuarios,Desarrollador,Clasificacion_por_Edad
0,WII SPORTS,Wii,2006-01-01,Deportes,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322,Nintendo,E
1,MARIO KART WII,Wii,2008-01-01,Carreras,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709,Nintendo,E
2,WII SPORTS RESORT,Wii,2009-01-01,Deportes,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192,Nintendo,E
3,NEW SUPER MARIO BROS.,DS,2006-01-01,Plataforma,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431,Nintendo,E
4,WII PLAY,Wii,2006-01-01,Miscelaniaelania,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129,Nintendo,E


In [21]:
#convierto la columna 'Año de lanzamiento' de int a datatime64
df['Año de lanzamiento'] = pd.to_datetime(df['Año de lanzamiento'], format='%Y')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7435 entries, 0 to 7434
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Juego                   7435 non-null   object        
 1   Consola                 7435 non-null   object        
 2   Año de lanzamiento      7435 non-null   datetime64[ns]
 3   Genero                  7435 non-null   object        
 4   Publicador              7433 non-null   object        
 5   Ventas USA              7435 non-null   float64       
 6   Ventas Europa           7435 non-null   float64       
 7   Ventas Japon            7435 non-null   float64       
 8   Ventas resto del Mundo  7435 non-null   float64       
 9   Ventas Totales          7435 non-null   float64       
 10  Puntuacion de criticos  6867 non-null   float64       
 11  Cantidad de criticos    6867 non-null   float64       
 12  Puntaje de Usuario      7435 non-null   float64 

In [27]:
print(df.Juego.value_counts())
print('\n',df.Consola.value_counts())
print('\n',df.Genero.value_counts())
print('\n',df.Publicador.value_counts())
print('\n',df.Desarrollador.value_counts())
print('\n',df.Clasificacion_por_Edad.value_counts())


Juego
NEED FOR SPEED: MOST WANTED    9
THE LEGO MOVIE VIDEOGAME       8
FIFA SOCCER 13                 8
ANGRY BIRDS STAR WARS          8
LEGO MARVEL SUPER HEROES       8
                              ..
NEW SUPER MARIO BROS. WII      1
WII PLAY                       1
NEW SUPER MARIO BROS.          1
WII SPORTS RESORT              1
MARIO KART WII                 1
Name: count, Length: 4632, dtype: int64

 Consola
PS2     1214
X360     944
PS3      859
PC       751
XB       569
Wii      545
DS       509
PSP      417
GC       355
PS4      256
GBA      251
XOne     182
3DS      171
PS       158
PSV      140
WiiU     100
DC        14
Name: count, dtype: int64

 Genero
Accion              1792
Deportes            1081
Disparos             900
RPG                  735
Carreras             629
Miscelaniaelania     428
Plataforma           419
Lucha                395
Simulacion           339
Estrategia           300
Aventura             292
Puzzle               125
Name: count, dtype: int64